# Iterative Prisoner's Dilemma


### Description

The [Prisoner's Dilemma](https://en.wikipedia.org/wiki/Prisoner%27s_dilemma) (PD) is a classical game analyzed in game theory, which is widely used to (attempt to) model social/economical interaction. It's a "dilemma" as, if exploited to explain the emergence of altruism in human or in general animal society, it fails badly at a first glance.

The classical situation-representation of the PD is that of two prisoners whose conviction depends on their mutual cooperation. It is easier understood though if illustrated in terms of a trade-off game (closed bag exachange):

*Two people meet and exchange closed bags, with the understanding that one of them contains money, and the other contains a purchase. Either player can choose to honor the deal by putting into his or her bag what he or she agreed, or he or she can defect by handing over an empty bag.*

It is obvious that for both players the winning strategy is to NOT cooperate.

Things changes when the interaction between the two individuals is iterated, in that case a more altruist attitude (strategy) is expected to emerge. The goal of this project is to test this hypothesis.

Mathematically the PD can be expressed with very basic linear algebra. The key component is the **Payoff matrix** $M$, which quantify the reward each player gets depending on whether she cooperated or not (defect):

$$
M = 
\begin{pmatrix} 
R & S \\
T & P 
\end{pmatrix}
$$

with $T,R,S,P$ integers that satisfy the following conditions:

$$
T>R>P>S; \quad 2R > T+S
$$

for example $T=3$, $R=2$, $P=1$ and $S=0$, or  $T=5$, $R=3$, $P=2$, $S=0$. Each player choice (move) can be represented by one of the two axis in ${\rm I\!R}^2$, i.e. $u_C=\begin{pmatrix} 1 \\ 0 \end{pmatrix}$ or $u_D=\begin{pmatrix} 0 \\ 1 \end{pmatrix}$, where the first coordinate stands for *Cooperate* and the second for *Defect*. Being $u_1$ and $u_2$ their rewards $r_1$ and $r_2$ can be computed then as:

$$
r_1 = u_1^T M u_2
\quad
\quad
r_2 = u_2^T M u_1
$$

In an Iterative Prisoner's Dilemma (IPD), two players play prisoner's dilemma more than once in succession and they remember previous actions of their opponent and change their strategy accordingly. The winning strategy is the one which yields to a larger reward at the end of the IPD.

The strategy can be represented as a function which outputs either $u_C$ or $u_D$. Such function can depend on the opponent's history of moves, her on history of moves, on the number of moves played till that moment and so on, but it can only be based on a probability density function. Possible strategies are:

* **Nice guy**: always cooperate (the function's output is always $u_D$)
* **Bad guy**: always defect 
* **Mainly nice**: randomly defect $k\%$ of the times and cooperate $100-k\%$, $k<50$
* **Mainly bad**: randomly defect $k\%$ of the times and cooperate $100-k\%$, $k>50$
* **tit-for-tat**: start by cooperating, then repeat what the opponent has done in the previous move 

Many more and much more complex strategies can be implemented. The strategy can even change during the IPD.


### Assignments

* Implement a simple IPD between two players implementing two given strategies. Study the evolution along the tournament confronting different strategies; study the overall outcome in the different configurations. 
* Implement a multiple players IPD (MPIPD) where several strategies play against each other in a roud-robin scheme
* Iterate what done in the previous task (repeated MPIPD, rMPIPD)  by increasing the population implementing a given strategy depending on the results that strategy achieved in the previous iteration
* (*difficult*) Implement a rMPIPD where strategies are allowed to mutate. The goal is to simulate the effect of genetic mutations and the effect of natura selection. A parameter (gene) should encode the attidue of an individual to cooperate, such gene can mutate randomly and the corresponding phenotype should compete in the MPIPD such that the best-fitted is determined.  


## Basic elements and 1-on-1 matches

In [1]:
import numpy as np
from numpy.random import random

We start by defining a `Player` class.

In [3]:
class Player:
    def __init__(self, k):
        self.k = k
    
    def play(self, hist):
        return random() < self.k
 
# special child class for tit-for-tat strategies (see later)
class TftPlayer(Player):
    def play(self, hist):
        return hist[-1]

Then a function that implements the basic 1-on-1 game mechanics. The default payoff matrix will be
$$
    M = \begin{pmatrix} 2 & 0 \\ 3 & 1 \end{pmatrix},
$$
corresponding to
$$
    R = 2,\, S = 0,\, T = 3,\, P = 1.
$$
If both players cooperate, they both receive 2 points (**R**eward). If they betray each other, they each receive 1 point (**P**unishment). If one player betrays their cooperative opponent, the betrayer receives 3 points (**T**emptation payoff) while the betrayed player gets 0 (**S**ucker’s payoff).

In [5]:
def game(pl1, pl2, it, payoff=(3, 2, 1, 0)):
    # initialize hists to true to correctly handle the first
    # iteration for a tit-for-tat player (needs to cooperate on
    # first move)
    hist1 = [True] * it
    hist2 = [True] * it
    points = [0, 0]
    
    for i in range(it):
        res1 = pl1.play(hist2)
        res2 = pl2.play(hist1)
        hist1[i] = res1
        hist2[i] = res2
        if res1 and res2:
            points[0] += payoff[1]
            points[1] += payoff[1]
        elif res1 and not res2:
            points[1] += payoff[0]
            points[0] += payoff[3]
        elif not res1 and res2:
            points[1] += payoff[3]
            points[0] += payoff[0]
        else:
            points[0] += payoff[2]
            points[1] += payoff[2]
 
    return points

Then, we define the following basic strategies:
* _nice guy_, a player who always cooperates;
* _bad guy_, a player who always betrays.

And the intermediate versions:
* _average guy_, a player who cooperates half the time;
* _mostly nice guy_, a player with a cooperation probability of 0.75;
* _mostly bad guy_, a player with a cooperation probability of 0.25.

Also, we’ll implement a basic example of a different kind of strategy, commonly referred to as “tit-for-tat”. A tit-for-tat player starts by cooperating on the first iteration and then copies their opponent’s previous move from that point forward.

In [ ]:
nice_guy = Player(1)
bad_guy = Player(0)
avg_guy = Player(0.5)
mn_guy = Player(0.75)
mb_guy = Player(0.25)
tt_guy = TftPlayer(0)